In [1]:
import sys
mins = 60

from hmmlearn import hmm

import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings
import sys
sys.path.append("../code/")
%matplotlib inline
warnings.filterwarnings("ignore")



ImportError: No module named 'hmmlearn'

In [2]:
from nilmtk import *
import os
import nilmtk
from nilmtk.disaggregate.fhmm_exact import sort_learnt_parameters
from nilmtk.disaggregate.fhmm_exact import create_combined_hmm
from nilmtk.disaggregate.fhmm_exact import FHMM

In [4]:
from dataloader import APPLIANCE_ORDER, get_train_test
num_folds =5
train, test = get_train_test(1, num_folds=num_folds, fold_num=0)



In [5]:
def learn_hmm_appliance(train_appliance, num_states):
    """
    train_appliance: M homes, D days, 24 hours
    """
    mod = hmm.GaussianHMM(num_states, "full")
    t = train_appliance.copy()
    t = t.reshape(-1, 24, 1)
    t = t[np.random.choice(t.shape[0], 400, replace=False), :, :]
    mod.fit(t)
    return mod


In [23]:
def learn_fhmm(train, num_states):
    models = {}
    for appliance_num, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        models[appliance] = learn_hmm_appliance(train[:, 1:, :, :][:, appliance_num, :, :], num_states)
        print(appliance)
#         print(models[appliance].means_)
#         print(" "*20)
    
    new_learnt_models = OrderedDict()
    for appliance, appliance_model in models.items():
        startprob, means, covars, transmat = sort_learnt_parameters(
                        appliance_model.startprob_, appliance_model.means_,
                        appliance_model.covars_, appliance_model.transmat_)
        new_learnt_models[appliance] = hmm.GaussianHMM(
                    startprob.size, "full", startprob, transmat)
        new_learnt_models[appliance].means_ = means
        new_learnt_models[appliance].covars_ = covars

    learnt_model_combined = create_combined_hmm(new_learnt_models)

    f = FHMM()
    f.model = learnt_model_combined
    f.individual = new_learnt_models
    return f

In [38]:
def make_fhmm_preds(num_states, dataset):
    out = []
    for cur_fold in range(5):
        print(cur_fold)
        print("------------")
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        f = learn_fhmm(train, num_states)
        print("Training done for fold")
        pred = np.zeros_like(test[:, 1:, :, :])
        for home in range(pred.shape[0]):
            for day in range(pred.shape[2]):
                pred_home_day = f.disaggregate_chunk(pd.Series(test[home, 0, day, :]))[APPLIANCE_ORDER[1:-1]].values.T
                for appliance_num in range(pred.shape[1]):
                    pred[home, appliance_num, day, :] = pred_home_day[appliance_num, :].flatten()
    
        out.append(pred)
    return np.concatenate(out)



In [45]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err ={}
gt = tensor[:, 1:, :, :]
pred = {}

for dataset in [1, 3]:
    err[dataset] = {}
    pred[dataset] = {}
    for num_states in range(2, 5):
        pred[dataset][num_states] = make_fhmm_preds(num_states, dataset)
        pred[dataset][num_states] = np.minimum(pred[dataset][num_states], tensor[:, 0:1, :,:])
        err[dataset][num_states] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[dataset][num_states][:, i,:,:].flatten(), 
                                                                           gt[:, i, :, :].flatten()) for i in range(pred[dataset][num_states].shape[1])}

0
------------
hvac
fridge
dr
dw
mw
Training done for fold
1
------------
hvac
fridge
dr
dw
mw
Training done for fold
2
------------
hvac
fridge
dr
dw
mw
Training done for fold
3
------------
hvac
fridge
dr
dw
mw
Training done for fold
4
------------
hvac
fridge
dr
dw
mw
Training done for fold
0
------------
hvac
fridge
dr
dw
mw
Training done for fold
1
------------
hvac
fridge
dr
dw
mw
Training done for fold
2
------------
hvac
fridge
dr
dw
mw
Training done for fold
3
------------
hvac
fridge
dr
dw
mw
Training done for fold
4
------------
hvac
fridge
dr
dw
mw
Training done for fold
0
------------
hvac
fridge
dr
dw
mw
Training done for fold
1
------------
hvac
fridge
dr
dw
mw
Training done for fold
2
------------
hvac
fridge
dr
dw
mw
Training done for fold
3
------------
hvac
fridge
dr
dw
mw
Training done for fold
4
------------
hvac
fridge
dr
dw
mw
Training done for fold
0
------------
hvac
fridge
dr
dw
mw
Training done for fold
1
------------
hvac
fridge
dr
dw
mw
Training done for fo

In [46]:
pd.DataFrame(err[3]).mean()

2    114.988576
3    170.482273
4    170.508034
dtype: float64

In [54]:
best_pred = {}
best_pred[1] = pred[1][2]
best_pred[3] = pred[3][2]

In [55]:
np.save("./fhmm_pred_1.npy", best_pred[1])
np.save("./fhmm_pred_3.npy", best_pred[3])